In [ ]:
import pandas as pd
import tabula
import numpy as np

In [ ]:
lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

vecteur = [lieux[0] if i < 4 else
           lieux[1] if i < 8 else
           lieux[2] if i < 13 else
           lieux[3] if i < 15 else
           lieux[4] if i < 17 else
           lieux[5] if i < 19 else
           lieux[6] if i < 29 else
           lieux[7] if i < 35 else
           lieux[8] if i < 48 else
           lieux[9] if i < 58 else
           lieux[10] if i < 63 else
           lieux[11] if i < 68 else
           lieux[12] if i < 74 else
           lieux[13] if i < 78 else
           lieux[14] if i < 79 else
           lieux[15] if i < 83 else
           'Valeur par défaut' for i in range(83)]


# Lien vers le PDF contenant le tableau
lien_pdf = "https://medias.paris2024.org/uploads/2022/07/Calendrier-par-epreuves-des-Jeux-Olympiques-de-Paris-2024.pdf"

try : 
    # Utilisation de tabula pour extraire le tableau du PDF
    # Utilisation de l'option 'pages' pour spécifier la page ou les pages contenant le tableau
    df_list = tabula.read_pdf(lien_pdf, pages=[1, 2], stream=True)  # Adapter les options selon vos besoins

    # Concaténer les DataFrames obtenus des différentes pages en un seul DataFrame
    df = pd.concat(df_list)

    # Supprimer les 28 dernières lignes
    df = df.iloc[:-32]
    df.drop(24, inplace=True)
    df.drop(23, inplace=True)
    df.drop(55, inplace=True)

    # Supprimer la première ligne du DataFrame
    df = df.drop(df.index[[0, 2]])  # Suppression de la première ligne
    df = df.drop(df.columns[[0]], axis=1)

    # Supprimer les lignes contenant uniquement des NaN
    df = df.dropna(how='all')
    
    df.columns = df.iloc[0]
    df.drop(1, inplace=True)

    df.index = vecteur
    
    df = df.replace('<', np.nan)

except Exception as e:
        print("Une erreur s'est produite :", str(e))




# %%
def format_date_horaire(cell, index_value):
    if isinstance(cell, str) and '-' in cell:
        date_month = '07' if int(index_value) in range(24, 32) else '08'
        year = '2024'
        
        date = f"{index_value}/{date_month}/{year}"
        horaire1, horaire2 = cell.split('-')
        
        return f"{date} {horaire1.strip()} - {horaire2.strip()}"
    else:
        return cell

# Appliquer la fonction à chaque cellule du DataFrame
for column in df.columns:
    df[column] = df.apply(lambda x: format_date_horaire(x[column], column), axis=1)


# %%
# Supprimer les titres des index
df.index.name = None

# Ajouter Lieux pour la première colonne
df.reset_index(inplace=True)
df.columns = ['Lieux'] + list(df.columns[1:])  # Renommer la première colonne en 'Lieux'

# %%
# Créer un dictionnaire pour stocker les valeurs non vides associées à chaque lieu
data = {}
for idx, row in df.iterrows():
    lieu = row['Lieux']
    if lieu not in data:
        data[lieu] = []

    # Récupérer les valeurs non vides et les ajouter à la liste des valeurs du lieu
    values = row.drop('Lieux').dropna().tolist()
    data[lieu].extend(values)

# Créer un nouveau DataFrame à partir du dictionnaire
new_df = pd.DataFrame.from_dict(data, orient='index')

lieux_a_supprimer = ['arena-paris-nord', 'site-escalade-bourget', 'centre-aquatique']

# Suppression des lignes spécifiées
new_df = new_df.drop(lieux_a_supprimer, axis=0)

# %%
# Fonction pour convertir le format de date
def convert_date_format(entry):
    if isinstance(entry, str):
        parts = entry.split(' ')
        if len(parts) == 5:
            date = parts[0]
            start_time = parts[1]
            return f"{date} {start_time}"
    return entry

# Appliquer la fonction sur toutes les colonnes du DataFrame
for col in new_df.columns:
    new_df[col] = new_df[col].apply(convert_date_format)


# %%
# Appliquer une fonction pour supprimer la partie après le tiret dans chaque cellule
horaires_début = new_df.applymap(lambda x: x.split(' -')[0] if isinstance(x, str) else x)

# Appliquer une fonction pour supprimer les 4 caractères avant le tiret dans chaque cellule
horaires_fin = new_df.applymap(lambda x: pd.Series(x).replace(r'.{5}\s-\s', '', regex=True)[0] if isinstance(x, str) else x)

horaires_début = horaires_début.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
horaires_début = horaires_début.dropna(axis=1, how='all')

horaires_fin = horaires_fin.applymap(lambda x: pd.to_datetime(x, errors='coerce'))
horaires_fin = horaires_fin.dropna(axis=1, how='all')

# Afficher le DataFrame résultant
display(horaires_début)
display(horaires_fin)


# %%
def adjust_time_moins_90min(cell):
    # Vérifier si la cellule est vide ou non
    if not pd.isnull(cell):
        # Convertir la cellule en objet datetime
        cell = pd.to_datetime(cell, format='%d/%m/%Y %H:%M')
        # Ajouter 1 heure et 30 minutes à l'objet datetime
        cell = cell - pd.Timedelta(hours=1, minutes=30)
    return cell

def adjust_time_plus_30min(cell):
    # Vérifier si la cellule est vide ou non
    if not pd.isnull(cell):
        # Convertir la cellule en objet datetime
        cell = pd.to_datetime(cell, format='%d/%m/%Y %H:%M')
        # Retirer 30 minutes à l'objet datetime
        cell = cell + pd.Timedelta(hours=0, minutes=30)
    return cell

# Parcourir chaque cellule du DataFrame et appliquer la fonction adjust_time
new_horaires_début_df = horaires_début.map(adjust_time_moins_90min)
new_horaires_fin_df = horaires_fin.map(adjust_time_plus_30min)

new_horaires_début_df.reset_index(inplace=True)
new_horaires_début_df.rename(columns={'index': 'Lieu'}, inplace=True)

new_horaires_fin_df.reset_index(inplace=True)
new_horaires_fin_df.rename(columns={'index': 'Lieu'}, inplace=True)

display(new_horaires_début_df)
display(new_horaires_fin_df)

new_horaires_fin_df.to_excel('new_horaires_fin_df.xlsx', index=False)
new_horaires_début_df.to_excel('new_horaires_début_df.xlsx', index=False)

# %%
def ajuster_plage_horaire(heure):
    if pd.notnull(heure):
        # Extraire la date et l'heure de la cellule
        date, time = str(heure).split(' ')
        heure, minute, seconde = map(int, time.split(':')[0:3])
        
        # Calculer la plage horaire correspondante
        plage_horaire = f"{heure:02d}:00:00 - {(heure+1)%24:02d}:00:00"
        
        # Construire la date avec la plage horaire
        return f"{date} {plage_horaire}"
    
    return heure

# Appliquer la fonction à toutes les colonnes (sauf la première)
new_horaires_début_df.iloc[:, 1:] = new_horaires_début_df.iloc[:, 1:].map(ajuster_plage_horaire)
new_horaires_fin_df.iloc[:, 1:] = new_horaires_fin_df.iloc[:, 1:].map(ajuster_plage_horaire)

display(new_horaires_début_df)
display(new_horaires_fin_df)


# %% [markdown]
# Ajout du script Nb de personnes + Arrêts + Lignes des lieux des événements

# %%
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

# %% [markdown]
# Liste des lieux et url 

# %%

lieux = [
'stade-tour-eiffel',
'arena-champ-de-mars',
'grand-palais',
'invalides',
'pont-alexandre-iii',
'trocadero', 
'concorde',
'stade-roland-garros',
'arena-paris-sud', 
'arena-bercy', 
'arena-la-chapelle', 
'paris-la-defense-arena', 
'centre-aquatique', 
'stade-de-france',
'site-escalade-bourget',
'arena-paris-nord'
]

urls = [
    f"https://www.paris2024.org/fr/site/{lieu.replace(' ', '-').lower()}/" for lieu in lieux
]

# %%
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_specific_lines(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        specific_lines = {
            'Capacité': [],
            'Stations': []  # Création d'une liste pour stocker les stations
        }
        for paragraph in soup.find_all('p'):
            # Recherche des lignes contenant les Capacité spécifiques
            if ('Capacité' in paragraph.text or 
                'Capacité court Philippe-Chatrier' in paragraph.text or 
                'Volleyball' in paragraph.text or 
                'Pour la gymnastique rythmique' in paragraph.text):
                specific_lines['Capacité'].append(paragraph.text.strip())
                
            # Extraction des phrases après "desservie" ou "desservi"
            if 'desservie' in paragraph.text or 'desservi' in paragraph.text:
                sentences = paragraph.text.split('.')  # Séparation en phrases
                for sentence in sentences:
                    if 'desservie' in sentence or 'desservi' in sentence:
                        specific_lines['Stations'].append(sentence.strip())
        
        return specific_lines
    return None


# Création d'un dictionnaire pour stocker les résultats
results = {lieu: get_specific_lines(url) for lieu, url in zip(lieux, urls)}

# Création d'un DataFrame à partir du dictionnaire
df = pd.DataFrame(results.items(), columns=['Lieu', 'Infos'])
# Expansion du dictionnaire pour créer les colonnes 'Capacité' et 'Stations'
df = pd.concat([df.drop(['Infos'], axis=1), df['Infos'].apply(pd.Series)], axis=1)


# %% [markdown]
# Compléter pour les données manquantes : Concorde 

# %%
# Faites une requête GET pour récupérer le contenu HTML de la page
url = 'https://sportetsociete.org/2022/10/20/paris-2024-un-cadre-majestueux-pour-louverture-des-jeux-paralympiques/#:~:text=Des%20tribunes%20de%2035%20000,des%20Champs%2DÉlysées%20pour%20saluer'
response = requests.get(url)

# Vérifiez si la requête a réussi
if response.status_code == 200:
    # Utilisation de BeautifulSoup pour analyser le contenu HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Recherchez le texte contenant "places"
    texte_places = soup.find(text=lambda text: text and "places" in text)

    if texte_places:
        # Imprimez le texte trouvé
        df.iloc[6, 1]=texte_places.strip()

# %% [markdown]
# Compléter pour les données manquantes : Arena Paris Sud

# %%
#Compléter pour les valeurs manquantes : arena-paris-sud
url = "https://fr.wikipedia.org/wiki/Arena_Porte_de_la_Chapelle"

# Obtenir le contenu de la page
response = requests.get(url)

if response.status_code == 200:
    # Analyser le contenu HTML avec BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Trouver le texte de la ligne commençant par "Capacité"
    capacity_text = soup.find("th", text="Capacité").find_next("td").get_text(strip=True)

    # Mettre à jour la case correspondante dans le DataFrame df
    df.iloc[8, 1] = capacity_text  # Ligne 8, colonne 2, indexées à partir de 0

else:
    print("La requête a échoué")

# %%
def extraire_nombres(texte):
    nombres = re.findall(r'[-+]?\d*\.\d+|\d+', str(texte))  # Expression régulière pour les nombres entiers ou décimaux
    return ' '.join(nombres)  # Retourne les nombres extraits sous forme d'une chaîne séparée par des espaces

# Appliquer la fonction sur la colonne 'Capacité'
df['Nombre de personnes à l\'événément'] = df['Capacité'].apply(extraire_nombres)

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer "202" par une chaîne vide dans la colonne 'Nombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace('202', '')

# Supprimer les espaces dans la colonne 'NNombre de personnes à l\'événément'
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].str.replace(' ', '')

# Supposons que 'df' est votre DataFrame contenant la colonne 'Nombres'

# Remplacer les valeurs dans la colonne 'Nombres' aux lignes spécifiques
df.loc[2, 'Nombre de personnes à l\'événément'] = '8000'  # Remplacer la deuxième ligne par '8000'
df.loc[10, 'Nombre de personnes à l\'événément'] = '6700'  # Remplacer la 10ème ligne par '6700'
df.loc[14, 'Nombre de personnes à l\'événément'] = '6000'  # Remplacer la 14ème ligne par '6000'

# Fonction pour supprimer le zéro en tête des nombres
def remove_leading_zero(number):
    if number.startswith('0'):
        return number[1:]
    return number

# Appliquer la fonction à la colonne 'Nombres' pour supprimer les zéros en tête
df['Nombre de personnes à l\'événément'] = df['Nombre de personnes à l\'événément'].astype(str).apply(remove_leading_zero)


# %%
# Boucle pour lire chaque ligne de la colonne 'Stations'
for index, station in df['Stations'].items():
    print(station)

# %%
arrêts=[['BIR HAKEIM', 'ALMA MARCEAU'],
        ['LA MOTTE-PICQUET-GRENELLE', 'LA MOTTE-PICQUET-GRENELLE', 'SEGUR'], 
        ['FRANKLIN D. ROOSEVELT', 'FRANKLIN D. ROOSEVELT'], 
        ['INVALIDES'], 
        ['INVALIDES'],
        ['IENA','TROCADERO'], 
        ['PALAIS-ROYAL','PALAIS-ROYAL','MADELEINE', 'MADELEINE', 'MADELEINE', 'OPERA'],
        ['PORTE DE SAINT-CLOUD','PORTE D\'AUTEUIL'], 
        ['PORTE DE VERSAILLES','BALARD','PORTE DE VANVES'], 
        ['BERCY','BERCY','GARE DE LYON'],
        ['PORTE DE LA CHAPELLE'],
        ['LA DEFENSE'],
        ['x'],
        ['SAINT-DENIS-PORTE DE PARIS'], 
        ['x'],
        ['x']]
 
df['Arrêts']=arrêts

# Fonction pour extraire les nombres après le mot "Métro"
def extract_numbers(text):
    matches = re.findall(r'Métro\s+(\d+)', str(text), flags=re.IGNORECASE)
    return matches

# Appliquer la fonction à la colonne 'Stations' pour créer une nouvelle colonne 'Nombres après Métro'
df['Lignes'] = df['Stations'].apply(extract_numbers)

# Compléter pour Franklin D
df.at[2, 'Lignes'] = [6, 9]

# %%
# Supprimer les colonnes 'Capacité' et 'Stations'
colonnes_a_supprimer = ['Capacité', 'Stations']
df = df.drop(colonnes_a_supprimer, axis=1)

# Calculer le nombre d'éléments dans chaque vecteur de la ligne 'Lignes'
df['Nombre de lignes'] = df['Lignes'].apply(lambda x: len(x) if isinstance(x, list) else None)

# Convertir la colonne 'Nombre de personnes à l\'événément' en type numérique (int ou float)
df['Nombre de personnes à l\'événément'] = pd.to_numeric(df['Nombre de personnes à l\'événément'], errors='coerce')

# Créer la nouvelle colonne en calculant la division
df['Nombre de personnes par ligne'] = df['Nombre de personnes à l\'événément'] / df['Nombre de lignes']

# Définir l'option d'affichage pour éviter la notation scientifique
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# %%
# Identifier les indices des lignes contenant les valeurs spécifiques dans la colonne 'lieu'
indices_a_supprimer = df[df['Lieu'].isin(['centre-aquatique', 'site-escalade-bourget', 'arena-paris-nord'])].index

# Supprimer les lignes correspondantes
df = df.drop(indices_a_supprimer)

display(df)

df.to_excel('df.xlsx', index=False)

# %% [markdown]
# Copie du programme fréquentation_metro

# %%
# Copie du programme Nettoyage données affluence par jour + nombre de validation par jour par station

# %%
import pandas as pd
%config NotebookApp.iopub_data_rate_limit=100000000000.0
url = '/workspaces/metro-JO/Données importées/2022_S2_PROFIL_FER.csv'
données_validation = pd.read_csv(url, delimiter=';')

# %%
# Convertir la colonne 'pourc_validations' en float (remplacer la virgule par un point pour séparer les décimales)
données_validation['pourc_validations'] = données_validation['pourc_validations'].str.replace(',', '.').astype(float)

# Grouper par 'LIBELLE_ARRET' et 'TRNC_HORR_60', puis calculer la somme des pourc_validations pour chaque groupe
tableau_somme_pour_validations = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()


# %%
# Mapping des tranches horaires à des heures spécifiques
tranche_horaire_to_hour = {
    '0H-1H': '00:00-01:00',
    '1H-2H': '01:00-02:00',
    '2H-3H': '02:00-03:00',
    '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00',
    '5H-6H': '05:00-06:00',
    '6H-7H': '06:00-07:00',
    '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00',
    '9H-10H': '09:00-10:00',
    '10H-11H': '10:00-11:00',
    '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00',
    '13H-14H': '13:00-14:00',
    '14H-15H': '14:00-15:00',
    '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00',
    '17H-18H': '17:00-18:00',
    '18H-19H': '18:00-19:00',
    '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00',
    '21H-22H': '21:00-22:00',
    '22H-23H': '22:00-23:00',
    '23H-24H': '23:00-00:00'
}

# Appliquer la conversion des tranches horaires au format heure
tableau_somme_pour_validations['TRNC_HORR_60'] = tableau_somme_pour_validations['TRNC_HORR_60'].map(tranche_horaire_to_hour)

# %%
# Regroupement des données en cas de doublons pour une paire 'LIBELLE_ARRET'-'TRNC_HORR_60'
grouped_data = données_validation.groupby(['LIBELLE_ARRET', 'TRNC_HORR_60'])['pourc_validations'].sum().reset_index()

# Pivotement des données
pivot_table = grouped_data.pivot(index='LIBELLE_ARRET', columns='TRNC_HORR_60', values='pourc_validations').reset_index()

# Renommage des colonnes
pivot_table.columns.name = None  # Supprimer le nom de la colonne index
pivot_table = pivot_table.rename(columns=lambda x: f"{x}" if x != 'LIBELLE_ARRET' else x)

# %%
import re

# Sélectionner les colonnes correspondant aux tranches horaires
horaire_columns = pivot_table.columns[1:]  # Exclure la colonne 'LIBELLE_ARRET'

# Fonction pour extraire l'heure de début de chaque tranche horaire
def extract_start_hour(column):
    match = re.match(r'(\d+)H-(\d+)H', column)
    if match:
        return int(match.group(1))
    return 0

# Trier les colonnes dans l'ordre croissant des heures de début des tranches
sorted_columns = sorted(horaire_columns, key=extract_start_hour)

# Réorganiser le tableau pivoté avec les colonnes dans l'ordre chronologique
new_column_order = ['LIBELLE_ARRET'] + sorted_columns
pivot_table = pivot_table[new_column_order]

# Supprimer la colonne 'ND' du DataFrame pivot_table_ordered
pivot_table.drop(columns=['ND'], inplace=True, errors='ignore')

pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.strip()  # Supprime les espaces en début et fin
pivot_table['LIBELLE_ARRET'] = pivot_table['LIBELLE_ARRET'].str.lower()  # Met tout en minuscules

# Créer un dictionnaire de correspondance pour renommer les colonnes
correspondance_heures = {
    '0H-1H': '00:00-01:00', '1H-2H': '01:00-02:00', '2H-3H': '02:00-03:00', '3H-4H': '03:00-04:00',
    '4H-5H': '04:00-05:00', '5H-6H': '05:00-06:00', '6H-7H': '06:00-07:00', '7H-8H': '07:00-08:00',
    '8H-9H': '08:00-09:00', '9H-10H': '09:00-10:00', '10H-11H': '10:00-11:00', '11H-12H': '11:00-12:00',
    '12H-13H': '12:00-13:00', '13H-14H': '13:00-14:00', '14H-15H': '14:00-15:00', '15H-16H': '15:00-16:00',
    '16H-17H': '16:00-17:00', '17H-18H': '17:00-18:00', '18H-19H': '18:00-19:00', '19H-20H': '19:00-20:00',
    '20H-21H': '20:00-21:00', '21H-22H': '21:00-22:00', '22H-23H': '22:00-23:00', '23H-0H': '23:00-00:00'
}

# Renommer les colonnes en utilisant le dictionnaire de correspondance
pivot_table.rename(columns=correspondance_heures, inplace=True)

# Afficher le DataFrame après suppression de la colonne 'ND'
print(pivot_table)